## SIMULAÇÃO DO CENÁRIO 1

IMPORTS

In [ ]:
import pandas as pd
import simpy
import os
import re
from unidecode import unidecode
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import numpy as np
import matplotlib.patches as mpatches

### ETAPA 1 - OBTENDO INFORMAÇÕES YAMAZUMI

In [ ]:
# OBTENDO TEMPOS POR ATIVIDADES DOS ARQUIVOS YAMAZUMI 
def get_process_times_from_csv(arquivo):
    """Lê um CSV Yamazumi, extrai os tempos por atividade e retorna um dicionário {atividade: tempo_em_segundos}."""
    
    def converter_tempo_para_segundos(tempo_str):
        if pd.isna(tempo_str) or not isinstance(tempo_str, str):
            return 0
        parts = str(tempo_str).split(':')
        try:
            if len(parts) == 3:
                h, m, s = map(int, parts)
                return h * 3600 + m * 60 + s
            elif len(parts) == 2:
                m, s = map(int, parts)
                return m * 60 + s
        except Exception:
            return 0
        return 0

    if not os.path.exists(arquivo):
        return {}

    df = pd.read_csv(arquivo, header=5, sep=';', encoding='latin1', on_bad_lines='skip')

    def normalizar_coluna(col):
        col = unidecode(col)
        col = col.upper()
        col = re.sub(r"[\"\'().\-\/]", "", col)
        col = re.sub(r"\s+", "_", col)
        return col.strip("_")

    df.columns = [normalizar_coluna(col) for col in df.columns]

    class_col = next((col for col in df.columns if 'CLASSIFICA' in col), None)
    if not class_col:
        return {}

    total_row = df[df[class_col] == 'Total'].copy()
    if total_row.empty:
        return {}

    start_col = df.columns.get_loc(class_col) + 1
    end_col = len(df.columns)
    try:
        end_col = df.columns.get_loc('COLUNA1')
    except KeyError:
        pass

    colunas_postos = df.columns[start_col:end_col]
    tempos_totais = total_row[colunas_postos].dropna(axis=1, how='all')
    tempos_formatados = tempos_totais.melt(var_name='Atividade', value_name='Tempo_str')
    tempos_formatados['Tempo_segundos'] = tempos_formatados['Tempo_str'].apply(converter_tempo_para_segundos)

    return pd.Series(tempos_formatados.Tempo_segundos.values, index=tempos_formatados.Atividade).to_dict()

### ETAPA 2: OBTENDO DADOS SEQUENCIA DO DIA

In [ ]:
sequencia = pd.read_csv("sequencia_producao.csv", sep=';')

# Extrai os 7 primeiros caracteres da coluna 'Baumuster'
sequencia['Baumuster_7dig'] = sequencia['Baumuster'].astype(str).str[:7]
sequencia_modelos = sequencia["Baumuster_7dig"].tolist()
print(sequencia_modelos)

### ETAPA 3: PREPARANDO DICIONÁRIO POR POSTO

In [ ]:
# --- Dicionários ---
#Adaptação para o cenário 1
MODELOS_CSV = {
    "Accelo": {"baumuster": ["C951102", "C951104", "C951111"], "perna": 1, "tempos": "Yamazumi - Accelo.csv"},
    "Atego": {"baumuster":["C951500", "C951511", "C951514"],"perna": 1, "tempos":"Yamazumi - Atego.csv"},
    "Atego 8x2 e 8x4": {"baumuster":[" C951530", " C951544"],"perna": 2, "tempos":"Yamazumi - Atego.csv"},
    "Atego (ATP)":{"baumuster": ["C968403", "C968114"], "perna": 2, "tempos":"Yamazumi - ATP.csv"}, 
    "Atego quinta roda": {"baumuster": [" C951501"], "perna": 2, "tempos":"Yamazumi - ATP.csv"},
    "Actros":{"baumuster": ["C963400", "C963403", "C963411", "C963414", "C963424", "C963425"], "perna": 2, "tempos":"Yamazumi - Actros.csv"},
    "Arocs": {"baumuster":["C964016", "C964216", "C964231", "C964416"], "perna": 2, "tempos":"Yamazumi - Arocs.csv"},
    "Axor (ATP +)":{"baumuster": ["C968150", "C968450", "C968453", "C968461", "C968475"], "perna":2, "tempos": "Yamazumi - Actros.csv"}
}

#Remoção do Atego (ATP)
ATIVIDADES_P1 = {'DIESEL':{'postos':['30A'],'modelos':['Accelo','Atego'],'operadores':1},
              'ARREFEC':{'postos':['31A'],'modelos':['Accelo','Atego'],'operadores':1},
              'REAPERTO':{'postos':['32A'],'modelos':['Accelo','Atego'],'operadores':1},
              '5AA_RODA':{'postos':['32C'],'modelos':['Accelo','Atego'],'operadores':1},
              'ESTEPE':{'postos':['32C'],'modelos':['Accelo','Atego'],'operadores':1},
              'APERTO_LE':{'postos':['32C'],'modelos':['Accelo','Atego'],'operadores':1},
              'PNEU_LD':{'postos':['33A'],'modelos':['Accelo','Atego'],'operadores':1}, 
              'PNEU_LE':{'postos':['33A'],'modelos':['Accelo','Atego'],'operadores':1},
              'APERTO_LD':{'postos':['34A'],'modelos':['Accelo','Atego'],'operadores':1},
              'GRADE':{'postos':['34A'],'modelos':['Accelo','Atego'],'operadores':1},
              'MECACNICA_1':{'postos':['34A','38'],'modelos':['Accelo','Atego'],'operadores':1},
              'MECACNICA_2':{'postos':['34A','38'],'modelos':['Accelo','Atego'],'operadores':1},
              'ELACTRICA_1':{'postos':['32A','32C','33A'],'modelos':['Accelo','Atego'],'operadores':1},
              'ELACTRICA_2':{'postos':['32A','32C','33A'],'modelos':['Accelo','Atego'],'operadores':1},
              'CONTROLE':{'postos':['34A'],'modelos':['Accelo','Atego'],'operadores':1},
              'MOTORISTA':{'postos':['38'],'modelos':['Accelo','Atego'],'operadores':1},
              'QUIS':{'postos':['38'],'modelos':['Accelo','Atego'],'operadores':1},
              }

#Adicionar novos carros que vão pra perna 2 e adaptar os postos da p1 paralelos a p2
ATIVIDADES_P2 = {'PASSADISASSO':{'postos':['30B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'ABASTECIMENTO':{'postos':['31B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'CHINELEIRA':{'postos':['32B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              '5AA_RODA': {'postos':['32D'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'PNEU_LD':{'postos':['33B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1}, 
              'PNEU_LE':{'postos':['33B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'PARALAMA_LD': {'postos':['34B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':0.5},#mesmo operador que o de aperto_ld e le
              'PARALAMA_LE':{'postos':['34B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':0.5},
              'CONTROLE': {'postos':['34B'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'ELACTRICA_I':{'postos':['33B','34B','39'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'ELACTRICA_II':{'postos':['33B','34B','39'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              'ELACTRICA_III':{'postos':['33B','34B','39'],'modelos':['Actros','Arocs', 'Axor (ATP +)'],'operadores':1},
              
              #Atividades da p1 que foram para a p2
              'DIESEL':{'postos':['30B'],'modelos':['Atego 8x2 e 8x4','Atego (ATP)', 'Atego quinta roda'],'operadores':1},
              'ARREFEC':{'postos':['31B'],'modelos':['Accelo','Atego'],'operadores':1},
              'REAPERTO':{'postos':['32B'],'modelos':['Accelo','Atego'],'operadores':1},
              '5AA_RODA':{'postos':['32D'],'modelos':['Accelo','Atego'],'operadores':1},
              'ESTEPE':{'postos':['32D'],'modelos':['Accelo','Atego'],'operadores':1},
              'APERTO_LE':{'postos':['32D'],'modelos':['Accelo','Atego'],'operadores':1},
              'PNEU_LD':{'postos':['33B'],'modelos':['Accelo','Atego'],'operadores':1}, 
              'PNEU_LE':{'postos':['33B'],'modelos':['Accelo','Atego'],'operadores':1},
              'APERTO_LD':{'postos':['34B'],'modelos':['Accelo','Atego'],'operadores':1},
              'GRADE':{'postos':['34B'],'modelos':['Accelo','Atego'],'operadores':1},
              'MECACNICA_1':{'postos':['34B','39'],'modelos':['Accelo','Atego'],'operadores':1},
              'MECACNICA_2':{'postos':['34A','39'],'modelos':['Accelo','Atego'],'operadores':1},
              'ELACTRICA_1':{'postos':['32B','32D','33B'],'modelos':['Accelo','Atego'],'operadores':1},
              'ELACTRICA_2':{'postos':['32B','32D','33B'],'modelos':['Accelo','Atego'],'operadores':1},
              'CONTROLE':{'postos':['34B'],'modelos':['Accelo','Atego'],'operadores':1},
              'MOTORISTA':{'postos':['39'],'modelos':['Accelo','Atego'],'operadores':1},
              'QUIS':{'postos':['39'],'modelos':['Accelo','Atego'],'operadores':1},
               }


In [ ]:
def gerar_dicionario_postos(MODELOS_CSV, ATIVIDADES_P1, ATIVIDADES_P2, get_process_times_from_csv):
    ordem_P1 = ['30A', '31A', '32A', '32C', '33A', '34A', '38']
    ordem_P2 = ['30B', '31B', '32B', '32D', '33B', '34B', '39']
    POSTOS = {}

    GRUPOS_ESPECIAIS = {
        'P1': {
            'PNEU': {"atividades": ['PNEU_LE', 'PNEU_LD'], "agreg": "media"},
            'APERTO': {"atividades": ['APERTO_LE', 'APERTO_LD'], "agreg": "media"},
            'MECACNICA': {"atividades": ['MECACNICA_1', 'MECACNICA_2'], "agreg": "media"},
            'ELETRICA': {"atividades": ['ELACTRICA_1', 'ELACTRICA_2'], "agreg": "media"}
        },
        'P2': {
            'PNEU': {"atividades": ['PNEU_LE', 'PNEU_LD'], "agreg": "media"},
            'PARALAMA': {"atividades": ['PARALAMA_LE', 'PARALAMA_LD'], "agreg": "media"},
            'ELETRICA': {"atividades": ['ELACTRICA_I', 'ELACTRICA_II', 'ELACTRICA_III'], "agreg": "media"}
        }
    }

    for modelo, dados_modelo in MODELOS_CSV.items():
        perna = dados_modelo["perna"]
        arquivo = dados_modelo["tempos"]
        tempos_atividade = get_process_times_from_csv(arquivo)
        
        print(tempos_atividade)
        
        atividades_dict = ATIVIDADES_P1 if perna == 1 else ATIVIDADES_P2
        ordem_postos = ordem_P1 if perna == 1 else ordem_P2
        grupos = GRUPOS_ESPECIAIS['P1'] if perna == 1 else GRUPOS_ESPECIAIS['P2']

        dicionario_postos = {}

        # Aplica regras especiais por grupo
        for nome_grupo, grupo in grupos.items():
            atividades = grupo["atividades"]

            if not all(a in tempos_atividade and a in atividades_dict for a in atividades):
                continue  # ignora se alguma atividade do grupo não existe

            if grupo["agreg"] == "media":
                tempo_total = sum(tempos_atividade[a] / atividades_dict[a]['operadores'] for a in atividades)
                tempo = tempo_total / len(atividades)
            elif grupo["agreg"] == "soma_oper":
                tempo_total = sum(tempos_atividade[a] for a in atividades)
                total_operadores = sum(atividades_dict[a]['operadores'] for a in atividades)
                tempo = tempo_total / total_operadores
            else:
                continue

            # aplica tempo dividido entre os postos das atividades
            postos_grupo = set(p for a in atividades for p in atividades_dict[a]['postos'])
            for posto in postos_grupo:
                dicionario_postos.setdefault(posto, {"tempo_tot": 0, "perna": perna})
                dicionario_postos[posto]["tempo_tot"] += tempo / len(postos_grupo)

        # adiciona atividades individuais
        for atividade, props in atividades_dict.items():
            if modelo not in props['modelos']:
                continue
            if atividade not in tempos_atividade:
                continue

            tempo = tempos_atividade[atividade] / props['operadores']
            for posto in props['postos']:
                dicionario_postos.setdefault(posto, {"tempo_tot": 0, "perna": perna})
                dicionario_postos[posto]["tempo_tot"] += tempo

        # ordena os postos
        postos_ordenados = {}
        for posto in ordem_postos:
            if posto in dicionario_postos:
                postos_ordenados[posto] = dicionario_postos[posto]

        POSTOS[modelo] = postos_ordenados

    return POSTOS

In [ ]:
POSTOS = gerar_dicionario_postos(MODELOS_CSV, ATIVIDADES_P1, ATIVIDADES_P2, get_process_times_from_csv)
print("\n--- Tempos por posto por modelo ---")
for modelo, valor in POSTOS.items():
    print(f"{modelo}: {valor}")

### ETAPA 4: FUNÇÃO PROCESSAMENTO DOS MODELOS

In [ ]:
TAKT_TIME=5.5*60
TURN_DURATION=14.5*3600

log_entrada_saida = [] # Lista vazia criada para guardar informações de entrada, saída e duração do modelo

# Função PROCESSAR_MODELO() simula a trajetória de um modelo pela linha de montagem:
# env é o ambiente de simulação do SimPy.
# nome é o nome único desse modelo na simulação, ex: "Actros_5".
# postos_dict é um dicionário com os postos que esse modelo vai passar, com tempos por posto.

def processar_modelo(env, nome, modelo_nome, perna, postos_dict, recursos):
    for posto, dados in postos_dict.items():
        tempo_tot = dados["tempo_tot"]
        with recursos[posto].request() as req: # pede o recurso do posto (garante que só um modelo usa por vez)
            entrada = env.now
            yield req #espera se necessário (yield req)
            yield env.timeout(tempo_tot) # espera o tempo de montagem (yield env.timeout(tempo_tot))
            saida = env.now

            # registra entrada, saída e duração no log
            log_entrada_saida.append({
                "modelo": modelo_nome,
                "nome": nome,
                "posto": posto,
                "entrada": entrada,
                "saida": saida,
                "duracao": saida - entrada,
                "perna": perna
            })

# Função SIMULACAO_LINHA cria o ambiente da simulação
# recursos: dicionário onde cada posto é um simpy.Resource() com capacity=1.
# ultimo_tempo_entrada_global: impede que dois modelos entrem no mesmo instante.
# disponibilidade_postos: controla quando o primeiro posto da perna estará livre para o próximo veículo.

def simulacao_linha(sequencia_modelos, MODELOS_CSV, resultado, TAKT_TIME, TURN_DURATION):

    env = simpy.Environment() # criando ambiente simulação
    
    # Ferramentas auxiliares na função:
    recursos = {} # dicionário onde cada posto é um simpy.Resource() com capacity=1;nenhum posto pode ser ocupado por mais de um modelo
    ultimo_tempo_entrada_global = 0  # ninguém entra na linha antes disso, impede que dois modelos entrem no mesmo instante nas pernas
    disponibilidade_postos = {}  # armazena quando o primeiro posto de cada perna estará livre para próximo veículo entrar

    # Função ALIMENTADOR() coloca os modelos na linha, respeitando o espaçamento (TAKT) e disponibilidade do primeiro posto.
    def alimentador(env):
        nonlocal ultimo_tempo_entrada_global

        # Identificando qual modelo corresponde ao baumuster da sequência e sua perna
        for i, baumuster in enumerate(sequencia_modelos):
            modelo_nome, perna = None, None
            for nome, dados in MODELOS_CSV.items():
                if baumuster in dados["baumuster"]:
                    modelo_nome = nome
                    perna = dados["perna"]
                    break
            if modelo_nome is None:
                continue
            
            # Define o primeiro posto que o modelo ocupará e quanto tempo levará nesse posto
            postos_dict = resultado[modelo_nome]
            primeiro_posto = next(iter(postos_dict))
            tempo_ciclo_primeiro_posto = postos_dict[primeiro_posto]["tempo_tot"]

            # Inicializa a disponibilidade se for a primeira vez usando esse posto
            if primeiro_posto not in disponibilidade_postos:
                disponibilidade_postos[primeiro_posto] = 0
            
            # só entra um modelo por vez no primeiro posto de cada perna;
            # o espaçamento entre modelos é de ao menos TAKT_TIME;
            # respeita o tempo mínimo em que o posto estará livre.

            # Espera até que seja possível entrar na linha
            tempo_disponivel_posto = disponibilidade_postos[primeiro_posto]
            entrada_sugerida = max(
                env.now,
                ultimo_tempo_entrada_global + TAKT_TIME,  # garante espaçamento global
                tempo_disponivel_posto  # garante que o primeiro posto da perna está livre
            )

            yield env.timeout(entrada_sugerida - env.now)

            # Atualiza os controles de entrada
            ultimo_tempo_entrada_global = entrada_sugerida
            disponibilidade_postos[primeiro_posto] = entrada_sugerida + tempo_ciclo_primeiro_posto

            # Cria os recursos dos postos, se necessário (caso não tenha sido criado)
            for posto in postos_dict:
                if posto not in recursos:
                    recursos[posto] = simpy.Resource(env, capacity=1)

            # Inicia processo de montagem
            env.process(processar_modelo(
                env=env,
                nome=f"{modelo_nome}_{i}",
                modelo_nome=modelo_nome,
                perna=perna,
                postos_dict=postos_dict,
                recursos=recursos
            ))
    
    # Inicia simulação, rodando até duração do turno
    env.process(alimentador(env))
    env.run(until=TURN_DURATION)
    return pd.DataFrame(log_entrada_saida)


### ETAPA 5: RESULTADOS SIMULAÇÃO

In [ ]:
# Executa a simulação
df_log = simulacao_linha(sequencia_modelos, MODELOS_CSV, POSTOS, TAKT_TIME, TURN_DURATION)

# Mostra todas as linhas (se quiser ver tudo)
pd.set_option('display.max_rows', None)

# Exibe o DataFrame com entradas e saídas por modelo/posto
print(df_log)

In [ ]:
def resumo_simulacao(df_log, TURN_DURATION):

    # 1. Quantidade de modelos produzidos por perna
    modelos_completos = df_log.groupby("nome").agg(
        perna=("perna", "first"),
        entrada_total=("entrada", "min"),
        saida_total=("saida", "max")
    )
    modelos_completos["tempo_total"] = modelos_completos["saida_total"] - modelos_completos["entrada_total"]
    producao_por_perna = modelos_completos["perna"].value_counts().sort_index()

    # 2. Tempo médio de produção por modelo por perna (em minutos)
    tempo_medio_producao = modelos_completos.groupby("perna")["tempo_total"].mean() / 60

    # 3. Takt time médio por perna (tempo médio por posto / número de postos)
    tempo_medio_por_posto = df_log.groupby(["perna", "posto"])["duracao"].mean()
    takt_time_medio = tempo_medio_por_posto.groupby("perna").mean() / 60  # em minutos

    # 4. Novo Takt time real por perna = TURN_DURATION / produção
    takt_time_real = (TURN_DURATION / producao_por_perna).sort_index() / 60  # em minutos

    # Exibindo os resultados
    print("Quantidade de modelos produzidos por perna:")
    for perna, qtd in producao_por_perna.items():
        print(f"  Perna {perna}: {qtd} modelos")

    print("\nTempo médio de produção de um modelo por perna (em minutos):")
    for perna, tempo in tempo_medio_producao.items():
        print(f"  Perna {perna}: {tempo:.2f} minutos")

    print("\nTempo médio de ocupação por posto (em minutos):")
    for perna, takt in takt_time_medio.items():
        print(f"  Perna {perna}: {takt:.2f} minutos")

    print("\nNovo Takt time real por perna (tempo disponível / modelos produzidos, em minutos):")
    for perna, takt in takt_time_real.items():
        print(f"  Perna {perna}: {takt:.2f} minutos")

# Exemplo de chamada
resumo_simulacao(df_log, TURN_DURATION)


### ETAPA 6: OUTPUTS GRÁFICOS  

In [ ]:
def plot_duracao_media_por_posto(df_log):
    """
    Gera gráficos de barras da duração média por posto para cada perna, com base em df_log.
    """
    df_log = df_log.copy()
    df_log["duracao_min"] = df_log["duracao"] / 60  # segundos → minutos

    for perna, cor in zip([1, 2], ["turquoise", "purple"]):
        media_perna = df_log[df_log["perna"] == perna].groupby("posto")["duracao_min"].mean()
        
        plt.figure(figsize=(10, 4))
        media_perna.plot(kind='bar', color=cor)
        plt.title(f"Duração média por posto - Perna {perna}")
        plt.ylabel("Duração (min)")
        plt.xlabel("Posto")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()

In [ ]:
plot_duracao_media_por_posto(df_log)

In [ ]:
def plot_tempo_atividades_por_modelo(MODELOS_CSV, ATIVIDADES_P1, ATIVIDADES_P2, get_process_times_from_csv):
    ordem_P1 = ['30A', '31A', '32A', '32C', '33A', '34A', '38']
    ordem_P2 = ['30B', '31B', '32B', '32D', '33B', '34B', '39']
    lista_dados = []

    # 1. Construir a lista de dados com tempos por modelo, atividade e posto
    for modelo, props in MODELOS_CSV.items():
        perna = props["perna"]
        arquivo = props["tempos"]
        atividades_dict = ATIVIDADES_P1 if perna == 1 else ATIVIDADES_P2
        ordem_postos = ordem_P1 if perna == 1 else ordem_P2

        tempos_atividade = get_process_times_from_csv(arquivo)
        
        if not tempos_atividade:
            print(f"Aviso: Nenhum tempo encontrado para {modelo} ({arquivo})")
            continue

        for atividade, dados in atividades_dict.items():
            if modelo not in dados["modelos"]:
                continue

            tempo_total = tempos_atividade.get(atividade, 0)
            tempo_por_operador = tempo_total / dados["operadores"] if dados["operadores"] > 0 else 0

            for posto in dados["postos"]:
                lista_dados.append({
                    "modelo": modelo,
                    "atividade": atividade,
                    "posto": posto,
                    "tempo_min": tempo_por_operador / 60,
                    "perna": perna
                })

    df_atividades = pd.DataFrame(lista_dados)

    def ordenar_por_posto(df, ordem_postos):
        df["ordem_posto"] = df["posto"].apply(lambda p: ordem_postos.index(p) if p in ordem_postos else -1)
        return df.sort_values(["ordem_posto", "atividade"])

    # 2. Gerar gráfico para cada modelo
    modelos = df_atividades["modelo"].unique()
    for modelo in modelos:
        df_modelo = df_atividades[df_atividades["modelo"] == modelo].copy()
        perna = df_modelo["perna"].iloc[0]
        ordem = ordem_P1 if perna == 1 else ordem_P2
        df_modelo = ordenar_por_posto(df_modelo, ordem)

        linha_meta = 5.5 if perna == 1 else 16.5

        plt.figure(figsize=(16, 7))
        sns.barplot(data=df_modelo, x="atividade", y="tempo_min", hue="posto", dodge=False)
        plt.axhline(y=linha_meta, color="red", linestyle="--", linewidth=1.5, label=f"Meta {linha_meta:.1f} min")
        plt.title(f"Tempo por Atividade - {modelo}")
        plt.xlabel("Atividade")
        plt.ylabel("Tempo por posto (minutos)")
        plt.xticks(rotation=45)
        plt.legend()
        plt.tight_layout()
        plt.show()

In [ ]:
plot_tempo_atividades_por_modelo(MODELOS_CSV, ATIVIDADES_P1, ATIVIDADES_P2, get_process_times_from_csv)

In [ ]:
def plot_modelos_por_perna(df_log):
    """
    Plota gráfico de barras com a quantidade de modelos únicos produzidos por perna.
    """
    modelos_por_perna = df_log.groupby("perna")["nome"].nunique()

    plt.figure(figsize=(6, 4))
    modelos_por_perna.plot(kind='bar', color='skyblue')
    plt.title("Modelos Produzidos por Perna")
    plt.xlabel("Perna")
    plt.ylabel("Quantidade de Modelos")
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_modelos_produzidos(df_log):
    """
    Plota gráfico de barras com a quantidade de veículos produzidos por tipo de modelo.
    """
    modelos_produzidos = df_log.groupby("modelo")["nome"].nunique().sort_values(ascending=False)

    plt.figure(figsize=(8, 5))
    modelos_produzidos.plot(kind='bar', color='navy')

    for i, valor in enumerate(modelos_produzidos):
        plt.text(i, valor + 0.2, f"{valor}", ha='center', va='bottom', fontsize=9)

    plt.title("Modelos Produzidos no Turno")
    plt.xlabel("Modelo")
    plt.ylabel("Quantidade de Veículos")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


In [ ]:
plot_modelos_por_perna(df_log)
plot_modelos_produzidos(df_log)

In [ ]:
def plot_eficiencia_postos_heatmap(df_log, tempo_turno_min=480):
    """
    Plota dois heatmaps (por perna) da eficiência de cada posto,
    definida como a média de tempo de uso dividido pelo tempo total disponível (por turno).

    Parâmetros:
    - df_log: DataFrame contendo colunas ['perna', 'posto', 'duracao']
    - tempo_turno_min: tempo total disponível por posto no turno, em minutos (default=480)
    """
    df_posto = df_log.copy()
    df_posto["duracao_min"] = df_posto["duracao"] / 60

    # Agrupar por perna e posto
    eficiencia_posto = df_posto.groupby(["perna", "posto"])["duracao_min"].mean().reset_index()
    eficiencia_posto["eficiencia"] = eficiencia_posto["duracao_min"] / tempo_turno_min

    # Pivotar para heatmap
    heatmap_p1 = eficiencia_posto[eficiencia_posto["perna"] == 1].pivot(index="posto", columns="perna", values="eficiencia")
    heatmap_p2 = eficiencia_posto[eficiencia_posto["perna"] == 2].pivot(index="posto", columns="perna", values="eficiencia")

    # Plot perna 1
    plt.figure(figsize=(4, 5))
    sns.heatmap(heatmap_p1, annot=True, fmt=".2%", cmap="YlGnBu", cbar=True)
    plt.title("Porcentagem de Ocupação por Posto - Perna 1 [%]")
    plt.ylabel("Posto")
    plt.xlabel("")
    plt.tight_layout()
    plt.show()

    # Plot perna 2
    plt.figure(figsize=(4, 5))
    sns.heatmap(heatmap_p2, annot=True, fmt=".2%", cmap="YlGnBu", cbar=True)
    plt.title("Porcentagem de Ocupação por Posto - Perna 2 [%]")
    plt.ylabel("Posto")
    plt.xlabel("")
    plt.tight_layout()
    plt.show()


In [ ]:
plot_eficiencia_postos_heatmap(df_log, tempo_turno_min=420)

In [ ]:
def plot_eficiencia_por_modelo_heatmap(df_log, TURN_DURATION):
 
    df = df_log.copy()
    df["duracao_min"] = df["duracao"] / 60

    # Agrupar por modelo, perna e posto
    eficiencia_modelo_posto = df.groupby(["perna", "modelo", "posto"])["duracao_min"].mean().reset_index()
    eficiencia_modelo_posto["eficiencia"] = (eficiencia_modelo_posto["duracao_min"] / TURN_DURATION)*100

    for perna in [1, 2]:
        data = eficiencia_modelo_posto[eficiencia_modelo_posto["perna"] == perna]

        # Obter todos os modelos e todos os postos da perna
        modelos_perna = df[df["perna"] == perna]["modelo"].unique()
        postos_perna = df[df["perna"] == perna]["posto"].unique()

        # Criar índice completo (todas combinações modelo × posto)
        full_index = pd.MultiIndex.from_product(
            [modelos_perna, postos_perna], names=["modelo", "posto"]
        )

        # Reindexar para garantir que todas as combinações estejam presentes
        data = data.set_index(["modelo", "posto"]).reindex(full_index).reset_index()
        data["eficiencia"] = data["eficiencia"].fillna(0)

        # Criar pivot para o heatmap
        heatmap_data = data.pivot(index="posto", columns="modelo", values="eficiencia")

        # Plotar
        plt.figure(figsize=(6, 5))
        sns.heatmap(heatmap_data, annot=True, fmt=".2%", cmap="RdPu", cbar=True)
        plt.title(f"Eficiência por Modelo e Posto - Perna {perna}")
        plt.ylabel("Posto")
        plt.xlabel("Modelo")
        plt.tight_layout()
        plt.show()


In [ ]:
plot_eficiencia_por_modelo_heatmap(df_log, TURN_DURATION)

In [ ]:
# Tempo total por modelo individual (nome), com identificação do tipo de modelo (ex: Accelo, Atego)
df_tempo_modelos = df_log.groupby("nome").agg(
    modelo=("modelo", "first"),
    tempo_total_min=("duracao", "sum")
).reset_index()
df_tempo_modelos["tempo_total_min"] = df_tempo_modelos["tempo_total_min"] / 60  # segundos → minutos

# Boxplot
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_tempo_modelos, x="modelo", y="tempo_total_min", palette="Set3")
plt.title("Distribuição do Tempo de Produção por Modelo")
plt.xlabel("Modelo")
plt.ylabel("Tempo Total de Produção (minutos)")
plt.xticks(rotation=45)
plt.show()

In [ ]:
modelos_completos = df_log.groupby("nome").agg(
    perna=("perna", "first"),
    entrada_total=("entrada", "min"),
    saida_total=("saida", "max")
)

for perna_id in sorted(modelos_completos["perna"].unique()):
    dados = modelos_completos[modelos_completos["perna"] == perna_id].copy()
    dados = dados.sort_values("saida_total")
    dados["cumulado"] = range(1, len(dados) + 1)

    plt.figure(figsize=(8, 4))
    plt.plot(dados["saida_total"] / 60, dados["cumulado"], marker='o')
    plt.title(f"Curva de Produção Cumulativa – Perna {perna_id}")
    plt.xlabel("Tempo (min)")
    plt.ylabel("Modelos Produzidos")
    plt.grid(True)
    plt.tight_layout()
    plt.show()


In [ ]:
from datetime import timedelta

def calcular_tempo_ocupado_real(intervalos):
    """Recebe lista de (entrada, saída) em segundos e retorna tempo total ocupado sem sobreposição (em minutos)"""
    if not intervalos:
        return 0

    # Ordena por entrada
    intervalos = sorted(intervalos, key=lambda x: x[0])
    tempo_ocupado = 0

    inicio, fim = intervalos[0]

    for i in range(1, len(intervalos)):
        prox_inicio, prox_fim = intervalos[i]
        if prox_inicio <= fim:
            fim = max(fim, prox_fim)  # sobrepõe
        else:
            tempo_ocupado += fim - inicio
            inicio, fim = prox_inicio, prox_fim

    tempo_ocupado += fim - inicio
    return tempo_ocupado / 60  # converte para minutos

tempo_total = 480  # 8 horas

# Coleta os intervalos de tempo (entrada, saída) por (posto, perna)
postos_intervalos = defaultdict(list)

for _, row in df_log.iterrows():
    chave = (row["perna"], row["posto"])
    postos_intervalos[chave].append((row["entrada"], row["saida"]))

# Calcula tempo real ocupado por posto
dados_inatividade = []

for (perna, posto), intervalos in postos_intervalos.items():
    tempo_ocupado = calcular_tempo_ocupado_real(intervalos)
    tempo_ocioso = max(0, tempo_total - tempo_ocupado)
    inatividade = tempo_ocioso / tempo_total

    dados_inatividade.append({
        "posto": posto,
        "perna": perna,
        "tempo_ocupado": tempo_ocupado,
        "inatividade": inatividade
    })

df_inatividade = pd.DataFrame(dados_inatividade)

# Gera gráfico por perna
for perna in [1, 2]:
    df_perna = df_inatividade[df_inatividade["perna"] == perna].copy()
    df_perna = df_perna.sort_values("posto")

    plt.figure(figsize=(10, 4))
    sns.barplot(data=df_perna, x="posto", y="inatividade", palette="Blues_r")
    plt.title(f"Inatividade por Posto - Perna {perna}")
    plt.ylabel("Inatividade relativa")
    plt.xlabel("Posto")
    plt.ylim(0, 1)
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    plt.tight_layout()
    plt.show()

In [ ]:
# Certifique-se de que entrada e saída estão em segundos
# tempo total de operação = 8 horas = 480 minutos
tempo_total = 480  # minutos

# Função para calcular tempo ocupado real, evitando sobreposição
def calcular_tempo_ocupado_real(intervalos):
    """Recebe lista de (entrada, saída) em segundos e retorna tempo total ocupado sem sobreposição (em minutos)"""
    if not intervalos:
        return 0

    intervalos = sorted(intervalos, key=lambda x: x[0])
    tempo_ocupado = 0

    inicio, fim = intervalos[0]
    for i in range(1, len(intervalos)):
        prox_inicio, prox_fim = intervalos[i]
        if prox_inicio <= fim:
            fim = max(fim, prox_fim)
        else:
            tempo_ocupado += fim - inicio
            inicio, fim = prox_inicio, prox_fim

    tempo_ocupado += fim - inicio
    return tempo_ocupado / 60  # em minutos

# Agrupar os intervalos por (perna, posto)
postos_intervalos = df_log.groupby(["perna", "posto"])[["entrada", "saida"]].apply(lambda x: list(x.itertuples(index=False, name=None)))

# Calcular inatividade
dados_inatividade = []
for (perna, posto), intervalos in postos_intervalos.items():
    tempo_ocupado = calcular_tempo_ocupado_real(intervalos)
    tempo_ocioso = max(0, tempo_total - tempo_ocupado)
    inatividade_pct = round(100 * tempo_ocioso / tempo_total, 2)
    
    dados_inatividade.append({
        "perna": perna,
        "posto": posto,
        "inatividade_%": inatividade_pct
    })

df_inatividade = pd.DataFrame(dados_inatividade)

# Criar heatmaps por perna
for perna in sorted(df_inatividade["perna"].unique()):
    df_p = df_inatividade[df_inatividade["perna"] == perna]
    df_pivot = df_p.pivot_table(index=["posto"], values="inatividade_%", aggfunc="mean")

    plt.figure(figsize=(6, len(df_pivot) * 0.5 + 1))
    sns.heatmap(df_pivot, annot=True, cmap="YlOrRd", fmt=".1f", cbar_kws={"label": "% de inatividade"})
    plt.title(f"Inatividade por Posto - Perna {perna}")
    plt.xlabel("")
    plt.ylabel("Posto")
    plt.tight_layout()
    plt.show()

In [ ]:
# Calcular o tempo de saída total por modelo (última estação)
modelos_completos = df_log.groupby("nome").agg(
    modelo=("modelo", "first"),
    saida_total=("saida", "max")
).reset_index()

# Criar colunas de slot de tempo de 30 minutos
modelos_completos["slot"] = (modelos_completos["saida_total"] // (30 * 60)).astype(int)

# Agrupar por slot e modelo e contar quantos modelos foram finalizados em cada slot
pivot = modelos_completos.groupby(["slot", "modelo"]).size().unstack(fill_value=0)

# Converter o índice (slot) para minutos desde o início do turno
pivot.index = pivot.index * 30

# Plot do gráfico empilhado
pivot.plot(kind="bar", stacked=True, figsize=(12, 5), colormap="tab20")
plt.xlabel("Minutos desde o início do turno")
plt.ylabel("Quantidade de Modelos Produzidos")
plt.title("Produção Empilhada por Modelo ao Longo do Turno")
plt.tight_layout()
plt.show()